<a href="https://colab.research.google.com/github/abdullahmujahidali/Vet-LangGraph/blob/main/VetAI_CustomPrompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q langchain langgraph langchain_openai openai pinecone langchain_community pymupdf

import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any, Optional, Literal
from datetime import datetime
from enum import Enum
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from pinecone import Pinecone
import base64
from google.colab import userdata
from dataclasses import dataclass
import json
import fitz

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)

index_name = "veterinary-ai-index"
embeddings = OpenAIEmbeddings()


class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str
    medical_records: Dict[str, Any]
    results: Dict[str, Any]
    metadata: Dict[str, Any]
    context: List[str]
    images: List[Dict[str, Any]]

In [21]:
@dataclass
class VetPrompts:
    """Collection of prompts for veterinary analysis agents"""

    BASE_VETERINARY_CONTEXT = """You are a veterinary specialist creating detailed medical documentation.
    CRITICAL REQUIREMENTS:
    - Document ALL numerical values with exact measurements and units
    - Record ALL dates in MM/DD/YY format with specific times when available
    - Use precise medical terminology without generalizations
    - Note EVERY abnormal finding with detailed descriptions
    - Include ALL diagnostic results with reference ranges
    - Document ALL medications with exact names, doses, frequencies, and routes
    - Record ALL client communications and instructions"""

    DATA_PROCESSOR = """You are a veterinary data processing agent conducting initial case review.

    REQUIRED DOCUMENTATION FORMAT:

    PATIENT SIGNALMENT & CLIENT INFORMATION:
    - Full patient details (ID#, name, species, breed, age, sex, weight to 0.1kg)
    - Complete client contact information
    - ALL current medications with exact doses/frequencies/routes
    - Insurance status and billing preferences

    PRESENTING COMPLAINT AND HISTORY:
    [Chief Complaint]
    - Exact duration of symptoms (days/weeks/months)
    - Precise progression timeline with specific dates
    - Detailed symptom frequency and severity patterns
    - ALL changes in behavior, appetite, drinking, urination, defecation

    [Past Medical History]
    - Complete chronological medical history with dates
    - ALL previous medications/treatments with responses
    - EVERY vaccination/preventive care date and product
    - ALL prior surgeries/procedures
    - Complete diet history with brands/amounts/frequencies

    PHYSICAL EXAMINATION:
    - ALL vital signs with exact values:
      * Temperature (°F to 0.1 degree)
      * Heart rate (beats/min)
      * Respiratory rate (breaths/min)
      * Weight (kg to 0.1)
      * Body condition score (1-9 scale)
      * Mucous membrane color
      * Capillary refill time (seconds)
      * Pain score (0-10 scale)

    - Detailed system-by-system examination:
      * EENT (including ALL abnormalities in eyes, ears, nose, throat)
      * Cardiovascular (heart sounds, pulses, rhythm)
      * Respiratory (breathing pattern, lung sounds, cough characteristics)
      * Gastrointestinal (including oral exam findings)
      * Musculoskeletal (gait, range of motion, pain responses)
      * Neurologic (detailed reflex and response testing)
      * Integumentary (ALL skin/coat abnormalities)
      * Lymphatic (ALL node sizes/characteristics)
      * Urogenital (including external genitalia examination)

    DIAGNOSTIC FINDINGS:
    - ALL laboratory results with reference ranges
    - EVERY imaging finding with measurements
    - ALL other test results with interpretations

    CURRENT MEDICATIONS AND TREATMENTS:
    - Complete current medication list with:
      * Generic and brand names
      * Exact doses (mg/kg or mg/lb)
      * Precise frequencies
      * Administration routes
      * Start/end dates
      * Noted effectiveness
      * ANY adverse reactions

    Note: Missing information must be explicitly marked as "Not Recorded" or "Not Performed" """

    HISTORY_ANALYZER = """You are a veterinary medical historian.

    REQUIRED ANALYSIS FORMAT:

    COMPREHENSIVE TIMELINE:
    1. Initial Disease Onset
       - Exact date of first clinical signs
       - Precise symptom progression
       - ALL initial diagnostic findings
       - First treatment responses

    2. Disease Progression
       - Detailed symptom evolution with dates
       - ALL complications or new symptoms
       - EVERY medication change with rationale
       - Treatment responses with specific timelines

    3. Complete Treatment History
       - ALL medications prescribed (chronological)
         * Generic and brand names
         * Exact doses and frequencies
         * Start and end dates
         * Specific responses to each
         * ANY adverse reactions

    4. Diagnostic Testing Chronology
       - ALL tests performed with dates
       - EVERY result with reference ranges
       - Specific clinical correlations
       - Changes in findings over time

    PATTERN ANALYSIS:
    1. Detailed Symptom Patterns
       - Frequency of episodes (exact numbers)
       - Severity trends (documented scale)
       - Specific triggers identified
       - Temporal patterns (time of day/season)

    2. Treatment Response Analysis
       - Duration of response to each intervention
       - Degree of improvement (percentage)
       - Treatment failures with specific reasons
       - Compliance assessment

    3. Risk Factor Evaluation
       - ALL identified risk factors
       - Environmental influences
       - Genetic predispositions
       - Concurrent conditions"""

    CLINICAL_ANALYZER = """You are a veterinary clinician performing detailed case analysis.

    REQUIRED ANALYSIS FORMAT:

    CLINICAL ASSESSMENT:
    1. Primary Clinical Signs Analysis
       - Individual assessment of EACH symptom:
         * Exact duration
         * Specific frequency
         * Severity scale (1-10)
         * Pattern of occurrence
         * Exacerbating/alleviating factors
         * Response to interventions

    2. Comprehensive Physical Examination Analysis
       - Vital Signs Interpretation:
         * Temperature (compare to 101.5±1°F normal)
         * Heart rate (compare to species normal)
         * Respiratory rate and effort
         * Weight trends (% change)
         * Body condition score changes

       - Detailed System Analysis:
         * EVERY abnormal finding with clinical significance
         * Specific measurements of all masses/lesions
         * Pain assessment scores
         * Comparative left vs. right findings
         * Sequential examination changes

    3. Diagnostic Testing Interpretation
       - Laboratory Results:
         * ALL values outside reference range
         * Trending changes in values
         * Clinical correlation of abnormalities
         * Impact on treatment decisions

       - Imaging Interpretation:
         * Specific anatomical findings
         * Measurements of structures
         * Comparison to prior studies
         * Clinical significance of changes

    4. Disease Process Analysis
       - Detailed Pathophysiology:
         * Primary disease mechanisms
         * Secondary complications
         * Organ system interactions
         * Disease stage with criteria
         * Progression indicators

    5. Treatment Response Evaluation
       - For EACH Medication:
         * Time to initial response
         * Degree of improvement
         * Duration of effect
         * Side effects observed
         * Factors affecting efficacy

    CLINICAL INTEGRATION:
    1. Evidence Synthesis
       - Supporting findings for each diagnosis
       - Strength of evidence assessment
       - Clinical trial relevance
       - Population-specific factors

    2. Contradictory Findings
       - Alternative explanations
       - Rule-out justifications
       - Confounding factors
       - Testing limitations

    Note: Address EVERY aspect above. Mark explicitly if data is unavailable."""

    DIAGNOSTICIAN = """You are a veterinary diagnostician providing specialist-level assessment.

    REQUIRED DOCUMENTATION FORMAT:

    PROBLEM LIST:
    1. Active Problems (prioritized)
       For EACH problem list:
       - Duration with exact onset date
       - Severity score (1-10)
       - Progression pattern
       - Related clinical findings
       - Impact on quality of life
       - Current treatment status

    2. Inactive/Resolved Problems
       For EACH problem list:
       - Original diagnosis date
       - Resolution date
       - Treatment provided
       - Risk of recurrence
       - Monitoring requirements

    DIFFERENTIAL DIAGNOSES:
    1. Primary Differential List
       For EACH differential:
       - Supporting evidence
       - Contradicting evidence
       - Diagnostic criteria met/unmet
       - Population prevalence
       - Age/breed predisposition
       - Required diagnostics

    2. Secondary Differential List
       For EACH differential:
       - Reason for lower priority
       - Required rule-out criteria
       - Testing recommendations

    DIAGNOSTIC PLAN:
    1. Immediate Diagnostics
       For EACH test:
       - Specific justification
       - Expected findings
       - Result interpretation criteria
       - Cost-benefit assessment
       - Sample handling requirements
       - Timing considerations

    2. Monitoring Diagnostics
       For EACH parameter:
       - Frequency of measurement
       - Alert thresholds
       - Trend analysis criteria
       - Action triggers

    THERAPEUTIC PLAN:
    1. Immediate Interventions
       For EACH treatment:
       - Drug name (generic/brand)
       - Exact dose calculation
       - Administration details
       - Monitoring requirements
       - Expected response timeline
       - Adverse effect monitoring

    2. Long-term Management
       - Specific outcome goals
       - Timeline for reassessment
       - Quality of life parameters
       - Client education points
       - Environmental modifications
       - Dietary recommendations

    All plans must include specific criteria for success/failure and explicit timeline for modifications."""

    SPECIALIST = """You are a veterinary specialist providing comprehensive case assessment.

    REQUIRED DOCUMENTATION FORMAT:

    SPECIALIST CONSULTATION:
    1. Case Summary
       - Detailed chronological history
       - ALL prior treatments with responses
       - EVERY diagnostic result
       - Specific clinical progression
       - Current status assessment

    2. Disease Process Analysis
       - Detailed pathophysiology
       - Current disease stage
       - Progression factors
       - Complicating conditions
       - Prognosis assessment with criteria

    3. Treatment Analysis
       - Response to previous treatments
       - Current treatment efficacy
       - Treatment complications
       - Compliance assessment
       - Risk-benefit analysis

    4. Specialist Recommendations
       For EACH recommendation:
       - Specific intervention
       - Scientific rationale
       - Expected outcome
       - Monitoring parameters
       - Alternative options

    COMPREHENSIVE PLAN:
    1. Medical Management
       For EACH medication:
       - Exact dose calculations
       - Administration protocol
       - Monitoring requirements
       - Duration of therapy
       - Success criteria
       - Modification triggers

    2. Monitoring Protocol
       For EACH parameter:
       - Measurement frequency
       - Specific target values
       - Acceptable ranges
       - Action thresholds
       - Response protocols

    3. Client Education
       - Disease process explanation
       - Treatment rationale
       - Administration instructions
       - Monitoring requirements
       - Warning signs
       - Emergency protocols

    4. Follow-up Schedule
       - Specific recheck dates
       - Testing schedule
       - Progress assessment criteria
       - Long-term monitoring plan
       - Quality of life assessment

    Documentation must be specific enough for any veterinarian to continue care."""

    CONNECTIONS = """You are a veterinary case integration specialist.

    REQUIRED INTEGRATION FORMAT:

    CASE SYNTHESIS:
    1. Disease Interactions
       - Primary-Secondary Disease Relationships:
         * Causal connections
         * Exacerbating factors
         * Systemic effects
         * Progressive impacts

       - Complicating Factors:
         * Concurrent conditions
         * Age-related factors
         * Breed predispositions
         * Environmental influences

       - Synergistic Effects:
         * Disease interactions
         * Treatment interactions
         * Cumulative impacts
         * Quality of life effects

    2. Treatment Integration
       - Drug Interactions:
         * Known interactions
         * Potential interactions
         * Monitoring requirements
         * Adjustment protocols

       - Therapeutic Conflicts:
         * Competing treatment needs
         * Priority assessment
         * Resolution strategies
         * Risk mitigation

       - Management Synergies:
         * Complementary treatments
         * Timing optimization
         * Efficiency improvements
         * Outcome enhancement

    3. Risk Assessment
       - Current Risk Factors:
         * Disease-specific risks
         * Treatment-related risks
         * Environmental risks
         * Client-specific factors

       - Potential Complications:
         * Early warning signs
         * Prevention strategies
         * Monitoring protocols
         * Intervention triggers

    INTEGRATED CARE PLAN:
    1. Comprehensive Treatment Strategy
       - Prioritized Interventions:
         * Immediate actions
         * Short-term goals
         * Long-term objectives
         * Success criteria

       - Coordinated Care:
         * Team responsibilities
         * Communication protocols
         * Documentation requirements
         * Quality assurance

    2. Long-term Management
       - Disease Trajectory:
         * Expected progression
         * Monitoring points
         * Intervention triggers
         * Outcome measures

       - Quality of Life:
         * Assessment criteria
         * Monitoring protocol
         * Improvement strategies
         * Client assessment tools

    3. Support Systems
       - Client Education:
         * Understanding assessment
         * Training needs
         * Resource requirements
         * Support services

       - Compliance Strategy:
         * Barriers assessment
         * Solution implementation
         * Monitoring methods
         * Success measures

    Integration must be comprehensive and actionable with clear responsibilities and timelines."""

VET_PROMPTS = VetPrompts()

In [22]:
def query_pinecone(query: str, top_k: int = 5) -> List[dict]:
    """Query Pinecone index for relevant context"""
    query_embedding = embeddings.embed_query(query)
    index = pc.Index(index_name)
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    return results.matches

def handle_error(state: VetState, error_message: str) -> VetState:
    new_state = state.copy()
    if "errors" not in new_state["results"]:
        new_state["results"]["errors"] = []
    new_state["results"]["errors"].append({
        "timestamp": datetime.now().isoformat(),
        "message": error_message,
        "context": state.get("context", [])
    })
    return new_state

def process_pdf(file_path: str) -> Tuple[List[str], List[Dict[str, Any]]]:
    """
    Process PDF document and extract both text and images
    Returns: Tuple of (text_chunks, image_data)
    """
    try:
        doc = fitz.open(file_path)
        text_chunks = []
        images = []

        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text()
            if text.strip():
                text_chunks.append(text)
            image_list = page.get_images()
            for img_index, img in enumerate(image_list):
                try:
                    xref = img[0]
                    base_image = doc.extract_image(xref)
                    image_data = {
                        'page_num': page_num + 1,
                        'image_index': img_index,
                        'extension': base_image["ext"],
                        'content': base64.b64encode(base_image["image"]).decode(),
                        'metadata': {
                            'width': base_image.get('width'),
                            'height': base_image.get('height'),
                            'colorspace': base_image.get('colorspace')
                        }
                    }
                    images.append(image_data)
                except Exception as e:
                    print(f"Error extracting image {img_index} from page {page_num + 1}: {str(e)}")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )

        if text_chunks:
            combined_text = "\n\n".join(text_chunks)
            final_chunks = text_splitter.split_text(combined_text)
        else:
            final_chunks = []

        doc.close()
        return final_chunks, images

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return [], []

def create_data_processing_agent():
    def process_data(state: VetState) -> VetState:
        try:
            relevant_docs = query_pinecone(state["current_input"])
            context = [doc.metadata.get("text", "") for doc in relevant_docs]

            if state.get("images"):
                image_context = "Document contains the following images:\n"
                for idx, img in enumerate(state["images"]):
                    image_context += f"\nImage {idx + 1} on page {img['page_num']}:\n"
                    image_context += f"- Type: {img['extension']}\n"
                    image_context += f"- Dimensions: {img['metadata']['width']}x{img['metadata']['height']}\n"
                context.append(image_context)

            context_text = "\n\n".join(context)

            prompt = ChatPromptTemplate.from_messages([
                ("system", VET_PROMPTS.BASE_VETERINARY_CONTEXT),
                ("system", VET_PROMPTS.DATA_PROCESSOR),
                ("human", "Process this veterinary case data in detail: {input}")
            ])

            messages = prompt.format_messages(
                input=state["current_input"],
                context=context_text
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["DataProcessor"] = response.content
            new_state["context"] = context

            return new_state

        except Exception as e:
            return handle_error(state, f"Data Processing Error: {str(e)}")

    return process_data


def create_history_analysis_agent():
    def analyze_history(state: VetState) -> VetState:
        try:
            processed_data = state["results"]["DataProcessor"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", VET_PROMPTS.BASE_VETERINARY_CONTEXT),
                ("system", VET_PROMPTS.HISTORY_ANALYZER),
                ("human", "Analyze this case history with detailed timeline: {processed_data}")
            ])

            messages = prompt.format_messages(
                processed_data=processed_data,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["HistoryAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"History Analysis Error: {str(e)}")

    return analyze_history

def create_clinical_analysis_agent():
    def analyze_clinical(state: VetState) -> VetState:
        try:
            history_analysis = state["results"]["HistoryAnalyzer"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", VET_PROMPTS.BASE_VETERINARY_CONTEXT),
                ("system", VET_PROMPTS.CLINICAL_ANALYZER),
                ("human", "Provide detailed clinical analysis for: {history_analysis}")
            ])

            messages = prompt.format_messages(
                history_analysis=history_analysis,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["ClinicalAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Clinical Analysis Error: {str(e)}")

    return analyze_clinical

def create_diagnostic_agent():
    def diagnose(state: VetState) -> VetState:
        try:
            clinical_analysis = state["results"]["ClinicalAnalyzer"]
            context = state.get("context", [])

            diagnostic_context = query_pinecone(clinical_analysis)
            additional_context = [doc.metadata.get("text", "") for doc in diagnostic_context]
            combined_context = context + additional_context

            prompt = ChatPromptTemplate.from_messages([
                ("system", VET_PROMPTS.BASE_VETERINARY_CONTEXT),
                ("system", VET_PROMPTS.DIAGNOSTICIAN),
                ("human", "Provide comprehensive diagnostic assessment for: {clinical_analysis}")
            ])

            messages = prompt.format_messages(
                clinical_analysis=clinical_analysis,
                context="\n\n".join(combined_context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["Diagnostics"] = response.content
            new_state["context"] = combined_context

            return new_state

        except Exception as e:
            return handle_error(state, f"Diagnostic Error: {str(e)}")

    return diagnose

def create_specialist_summary_agent():
    def generate_summaries(state: VetState) -> VetState:
        try:
            diagnostic_assessment = state["results"]["Diagnostics"]
            context = state.get("context", [])
            prompt = ChatPromptTemplate.from_messages([
                ("system", VET_PROMPTS.BASE_VETERINARY_CONTEXT),
                ("system", VET_PROMPTS.SPECIALIST),
                ("human", "Generate comprehensive specialist assessment for: {diagnostic_assessment}")
            ])
            messages = prompt.format_messages(
                diagnostic_assessment=diagnostic_assessment,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["SpecialistSummary"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Specialist Summary Error: {str(e)}")

    return generate_summaries

def create_connections_agent():
    def analyze_connections(state: VetState) -> VetState:
        try:
            previous_analyses = {
                "data_processing": state["results"]["DataProcessor"],
                "history": state["results"]["HistoryAnalyzer"],
                "clinical": state["results"]["ClinicalAnalyzer"],
                "diagnostics": state["results"]["Diagnostics"],
                "specialist": state["results"]["SpecialistSummary"]
            }

            context = state.get("context", [])
            prompt = ChatPromptTemplate.from_messages([
                ("system", VET_PROMPTS.BASE_VETERINARY_CONTEXT),
                ("system", VET_PROMPTS.CONNECTIONS),
                ("human", "Provide comprehensive case integration analysis for: {previous_analyses}")
            ])

            messages = prompt.format_messages(
                previous_analyses=json.dumps(previous_analyses, indent=2),
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["Connections"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Connections Analysis Error: {str(e)}")

    return analyze_connections

def create_workflow():
    """Create and configure the workflow"""
    workflow = StateGraph(VetState)


    workflow.add_node("process_data", create_data_processing_agent())
    workflow.add_node("analyze_history", create_history_analysis_agent())
    workflow.add_node("analyze_clinical", create_clinical_analysis_agent())
    workflow.add_node("diagnose", create_diagnostic_agent())
    workflow.add_node("specialist_summary", create_specialist_summary_agent())
    workflow.add_node("analyze_connections", create_connections_agent())

    workflow.set_entry_point("process_data")

    workflow.add_edge("process_data", "analyze_history")
    workflow.add_edge("analyze_history", "analyze_clinical")
    workflow.add_edge("analyze_clinical", "diagnose")
    workflow.add_edge("diagnose", "specialist_summary")
    workflow.add_edge("specialist_summary", "analyze_connections")
    workflow.add_edge("analyze_connections", END)

    return workflow.compile()

def analyze_veterinary_case(file_path: str):
    """Main function to analyze a veterinary case"""
    try:
        print("Processing document...")
        text_chunks, images = process_pdf(file_path)

        initial_state = {
            "messages": [],
            "current_input": "\n\n".join(text_chunks),
            "medical_records": {},
            "results": {
                "DataProcessor": "",
                "HistoryAnalyzer": "",
                "ClinicalAnalyzer": "",
                "Diagnostics": "",
                "SpecialistSummary": "",
                "Connections": ""
            },
            "metadata": {},
            "context": []
        }

        print("Creating workflow...")
        app = create_workflow()

        print("Running analysis...")
        result = app.invoke(initial_state)

        return result

    except Exception as e:
        print(f"Error in analysis: {str(e)}")
        raise

In [23]:
    file_path = "./input.pdf"
    result = analyze_veterinary_case(file_path)

    print("\nAnalysis Results:")
    print("-" * 50)

    sections = [
        ("Data Processing", "DataProcessor"),
        ("History Analysis", "HistoryAnalyzer"),
        ("Clinical Analysis", "ClinicalAnalyzer"),
        ("Diagnostic Assessment", "Diagnostics"),
        ("Specialist Summary", "SpecialistSummary"),
        ("Connections Analysis", "Connections")
    ]

    for title, key in sections:
        print(f"\n{title}:")
        print("-" * 30)
        print(result["results"].get(key, "Not available"))

    if "errors" in result["results"]:
        print("\nErrors encountered:")
        for error in result["results"]["errors"]:
            print(f"- {error['message']} (at {error['timestamp']}")

Processing document...
Creating workflow...
Running analysis...

Analysis Results:
--------------------------------------------------

Data Processing:
------------------------------
**PATIENT SIGNALMENT & CLIENT INFORMATION:**
- Patient: Jax Miracle
- Patient ID: CA2441B8
- Species: Canine
- Breed: Pomeranian
- Age: 4 years old
- Sex: Male, neutered
- Weight: 6.1 kg
- Microchip: None
- Client: Anna Miracle
- Client ID: 24404F
- Address: 2497 Hillsboro Rd, Campbellsburg, KY 40011, USA
- Phone: (502) 532-6641
- Email: miracletricia@gmail.com

**PRESENTING COMPLAINT AND HISTORY:**
- Chief Complaint: Coughing
- Duration: Acute onset
- Symptom Description: Coughing, hacking, panting loudly
- No vomiting or diarrhea reported
- Dry cough observed
- No changes in drinking, urination, or appetite noted
- No new lumps or bumps observed
- No recent travel outside Kentucky/Southern IN region
- On heartworm prevention
- Diet: Variety, Pedigree canned food, Kibbles and Bits dry food, Dreambones for